This code is from HuggingFace

In [1]:
import ollama

EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

# Each element in the VECTOR_DB will be a tuple (chunk, embedding)
# The embedding is a list of floats, for example: [0.1, 0.04, -0.34, 0.21, ...]
VECTOR_DB = []

def add_chunk_to_database(chunk):
  embedding = ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
  VECTOR_DB.append((chunk, embedding))


In [15]:
dataset = []
with open('test.txt', 'r',encoding="utf-8") as file:
  dataset = file.readlines()
  print(f'Loaded {len(dataset)} entries')


Loaded 7 entries


In [16]:
for i, chunk in enumerate(dataset):
  add_chunk_to_database(chunk)
  print(f'Added chunk {i+1}/{len(dataset)} to the database')


Added chunk 1/7 to the database
Added chunk 2/7 to the database
Added chunk 3/7 to the database
Added chunk 4/7 to the database
Added chunk 5/7 to the database
Added chunk 6/7 to the database
Added chunk 7/7 to the database


In [17]:
def cosine_similarity(a, b):
  dot_product = sum([x * y for x, y in zip(a, b)])
  norm_a = sum([x ** 2 for x in a]) ** 0.5
  norm_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (norm_a * norm_b)


In [18]:
def retrieve(query, top_n=3):
  query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=query)['embeddings'][0]
  # temporary list to store (chunk, similarity) pairs
  similarities = []
  for chunk, embedding in VECTOR_DB:
    similarity = cosine_similarity(query_embedding, embedding)
    similarities.append((chunk, similarity))
  # sort by similarity in descending order, because higher similarity means more relevant chunks
  similarities.sort(key=lambda x: x[1], reverse=True)
  # finally, return the top N most relevant chunks
  return similarities[:top_n]


In [26]:
input_query = input('Ask me a question: ')
retrieved_knowledge = retrieve(input_query)

print('Retrieved knowledge:')
for chunk, similarity in retrieved_knowledge:
  print(f' - (similarity: {similarity:.2f}) {chunk}')

instruction_prompt = f'''You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
{'\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])}
'''


Retrieved knowledge:
 - (similarity: 0.58) I can ’ t believe it ! __eou__ What ’ s wrong ? That was a great goal . __eou__ Yes , but I bet $ 200 dollars on the Cougars ! __eou__ Looks like you ’ re going to lose out on this game then . __eou__ I can ’ t believe it ! I thought the Cougars were going to win for sure . __eou__ What were the odds ? __eou__ 20 to 1 , in favour of the Cougars ! __eou__ Too bad . __eou__

 - (similarity: 0.58) Merry Christmas , Linda ! __eou__ Merry Christmas , Lee ! __eou__ Linda , thank you for this wonderful Christmas dinner . I really enjoyed it . __eou__ Thank you for coming . I'm glad you liked it . __eou__

 - (similarity: 0.57) Merry Christmas , Bill ! __eou__ Merry Christmas . Steven ! __eou__ What do you want to do to celebrate Christmas ? __eou__ I'll have supper with my girlfriend and go shopping . What about you ? __eou__ I'll go to church first and then go to a Christmas party . Would you like to come ? __eou__ I'd love to . But I am going to me

```
    ollama pull hf.co/CompendiumLabs/bge-base-en-v1.5-gguf
    ollama pull hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF
```
